In [3]:
#!/usr/bin/env python
# coding: utf-8
import os
import streamlit as st
import sqlite3
import langchain
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util

In [2]:
import os
print(os.getcwd())
os.chdir(r"c:\Users\Sanju\LangChain")

c:\Users\Sanju\LangChain


In [5]:
# Create a Streamlit app
#st.title("🤖 ChatDoc: Your Document Assistant")
st.markdown("<h1 style='text-align: left; color: #0000CD; font-size: 34px;'>🤖 ChatDoc: Your Document Assistant</h1>", unsafe_allow_html=True)
#st.subheader("Ask questions about your documents and get instant answers!")

# Relative paths to the 'docs' and 'data' folders in the GitHub repository
DOCS_DIRECTORY = "Epic"
PERSIST_DIRECTORY = "chroma"

# Create a place to input the OpenAI API Key
openai_api_key = st.text_input("Enter your OpenAI API Key", type="password")

# Document loading and processing
documents = []
document_names = []

if openai_api_key:
    # Set the OpenAI API key if it has been provided
    os.environ['OPENAI_API_KEY'] = openai_api_key

    for file in os.listdir(DOCS_DIRECTORY):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(DOCS_DIRECTORY, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
            document_names.append(os.path.splitext(file)[0])
        elif file.endswith('.docx') or file.endswith('.doc'):
            doc_path = os.path.join(DOCS_DIRECTORY, file)
            loader = Docx2txtLoader(doc_path)
            documents.extend(loader.load())
            document_names.append(os.path.splitext(file)[0])
        elif file.endswith('.txt'):
            text_path = os.path.join(DOCS_DIRECTORY, file)
            loader = TextLoader(text_path)
            documents.extend(loader.load())
            document_names.append(os.path.splitext(file)[0])

    # Create a RecursiveCharacterTextSplitter object
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=10, separators=["\n\n", "\n", "(?<=\. )", " ", ""])
    # Split the list of documents
    documents = text_splitter.split_documents(documents)

In [6]:
hugging_face_model_name = st.text_input("Enter your Hugging Face model name", value="bert-base-uncased")
embedding = HuggingFaceEmbeddings(model_name=hugging_face_model_name)

No sentence-transformers model found with name C:\Users\Sanju/.cache\torch\sentence_transformers\bert-base-uncased. Creating a new one with MEAN pooling.


In [7]:
vectordb = Chroma.from_documents(documents, embedding=embedding)

ValueError: Expected IDs to be a non-empty list, got []